### 목표 0: 음향 데이터 파일 불러오기, 평균 재생시간 알아보기

In [3]:
import os

# Pandas
import pandas as pd

# Scikit learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils import class_weight

# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical


# Audio
import librosa
import librosa.display

# Plot
import matplotlib.pyplot as plt

# Utility
import glob
import numpy as np
from tqdm import tqdm
import IPython.display as ipd
from timeit import default_timer as timer

import soundfile as sf

In [4]:
df = pd.read_csv('event_data_preprocesssing_jjin_real_final.csv')
df

,Unnamed: 0,waveFile,code,situation,no,event,eventCount,from,to,file_name
0,0,code0_dating-violence_01.TextGrid,code0,dating-violence,1.0,footstep,1.0,5.79780,9.44507,code0_dating-violence_01.wav
1,1,code0_dating-violence_01.TextGrid,code0,dating-violence,1.0,door,1.0,9.51020,10.35688,code0_dating-violence_01.wav
2,2,code0_dating-violence_01.TextGrid,code0,dating-violence,1.0,slang,1.0,17.71654,25.92289,code0_dating-violence_01.wav
3,3,code0_dating-violence_01.TextGrid,code0,dating-violence,2.0,groan,1.0,26.37880,34.97592,code0_dating-violence_01.wav
4,4,code0_dating-violence_01.TextGrid,code0,dating-violence,1.0,groan,1.0,37.06007,37.19033,code0_dating-violence_01.wav
...,...,...,...,...,...,...,...,...,...,...
15850,15853,normal_2(time8100to8400).TextGrid,code5,normal,7.0,male_speech,1.0,109.47259,110.95121,code5_normal_2(time8100to8400).wav
15851,15854,normal_2(time8100to8400).TextGrid,code5,normal,8.0,footstep,1.0,111.89214,112.29540,code5_normal_2(time8100to8400).wav
15852,15855,normal_2(time8100to8400).TextGrid,code5,normal,3.0,footstep,1.0,113.33131,118.34427,code5_normal_2(time8100to8400).wav
15853,15856,normal_2(time8100to8400).TextGrid,code5,normal,4.0,footstep,1.0,119.01636,123.04894,code5_normal_2(time8100to8400).wav


In [5]:
train_path = "./AnnotatedAudioFiles/"
train_file_list = os.listdir(train_path)
train_file_names = [file for file in train_file_list if file.endswith(".wav")]
train_file_names.sort(key=lambda x: (x.partition('.')[0]))
print(train_file_names)

['code0_dating-violence_01.wav', 'code0_dating-violence_02.wav', 'code0_dating-violence_03.wav', 'code0_dating-violence_04.wav', 'code0_dating-violence_05.wav', 'code0_dating-violence_06.wav', 'code0_dating-violence_07.wav', 'code0_dating-violence_08.wav', 'code0_dating-violence_09.wav', 'code0_dating-violence_10.wav', 'code0_dating-violence_11.wav', 'code0_dating-violence_12.wav', 'code0_dating-violence_13.wav', 'code0_dating-violence_14.wav', 'code0_dating-violence_15.wav', 'code0_dating-violence_16.wav', 'code0_dating-violence_17.wav', 'code0_dating-violence_18.wav', 'code0_dating-violence_19.wav', 'code0_dating-violence_20.wav', 'code0_dating-violence_21.wav', 'code0_dating-violence_22.wav', 'code0_dating-violence_23.wav', 'code0_dating-violence_24.wav', 'code0_dating-violence_25.wav', 'code0_dating-violence_26.wav', 'code0_dating-violence_27.wav', 'code0_dating-violence_28.wav', 'code0_dating-violence_29.wav', 'code0_dating-violence_30.wav', 'code0_dating-violence_A01.wav', 'code0

In [6]:
j = len(train_file_names)
print(j)

1672


In [21]:
for i in range(j):
    file_path = './AnnotatedAudioFiles/' + train_file_names[i]
    print(file_path)
    x, sr = sf.read(file_path, always_2d=True)
    start = int(5 * sr)
    end = int(35 * sr)
    y = x[start : end]
    sf.write(f'CNN_all/cut_from5_to35/{train_file_names[i]}', y, sr)

./AnnotatedAudioFiles/code0_dating-violence_01.wav
./AnnotatedAudioFiles/code0_dating-violence_02.wav
./AnnotatedAudioFiles/code0_dating-violence_03.wav
./AnnotatedAudioFiles/code0_dating-violence_04.wav
./AnnotatedAudioFiles/code0_dating-violence_05.wav
./AnnotatedAudioFiles/code0_dating-violence_06.wav
./AnnotatedAudioFiles/code0_dating-violence_07.wav
./AnnotatedAudioFiles/code0_dating-violence_08.wav
./AnnotatedAudioFiles/code0_dating-violence_09.wav
./AnnotatedAudioFiles/code0_dating-violence_10.wav
./AnnotatedAudioFiles/code0_dating-violence_11.wav
./AnnotatedAudioFiles/code0_dating-violence_12.wav
./AnnotatedAudioFiles/code0_dating-violence_13.wav
./AnnotatedAudioFiles/code0_dating-violence_14.wav
./AnnotatedAudioFiles/code0_dating-violence_15.wav
./AnnotatedAudioFiles/code0_dating-violence_16.wav
./AnnotatedAudioFiles/code0_dating-violence_17.wav
./AnnotatedAudioFiles/code0_dating-violence_18.wav
./AnnotatedAudioFiles/code0_dating-violence_19.wav
./AnnotatedAudioFiles/code0_dat

In [20]:
for i in range(j):
    file_path = './AnnotatedAudioFiles/' + train_file_names[i]
    print(file_path)
    x, sr = sf.read(file_path, always_2d=True)
    start = int(5 * sr)
    end = int(40 * sr)
    y = x[start : end]
    sf.write(f'CNN_all/cut_from5_to40/{train_file_names[i]}', y, sr)

./AnnotatedAudioFiles/code0_dating-violence_01.wav
./AnnotatedAudioFiles/code0_dating-violence_02.wav
./AnnotatedAudioFiles/code0_dating-violence_03.wav
./AnnotatedAudioFiles/code0_dating-violence_04.wav
./AnnotatedAudioFiles/code0_dating-violence_05.wav
./AnnotatedAudioFiles/code0_dating-violence_06.wav
./AnnotatedAudioFiles/code0_dating-violence_07.wav
./AnnotatedAudioFiles/code0_dating-violence_08.wav
./AnnotatedAudioFiles/code0_dating-violence_09.wav
./AnnotatedAudioFiles/code0_dating-violence_10.wav
./AnnotatedAudioFiles/code0_dating-violence_11.wav
./AnnotatedAudioFiles/code0_dating-violence_12.wav
./AnnotatedAudioFiles/code0_dating-violence_13.wav
./AnnotatedAudioFiles/code0_dating-violence_14.wav
./AnnotatedAudioFiles/code0_dating-violence_15.wav
./AnnotatedAudioFiles/code0_dating-violence_16.wav
./AnnotatedAudioFiles/code0_dating-violence_17.wav
./AnnotatedAudioFiles/code0_dating-violence_18.wav
./AnnotatedAudioFiles/code0_dating-violence_19.wav
./AnnotatedAudioFiles/code0_dat